In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAI
model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
output = model.invoke("自己紹介してください。")
print(output)



私は山田太郎と申します。東京出身で、現在は大学生として都内の大学に通っています。趣味は読書や映画鑑賞、旅行などです。また、大学では国際関係を専攻しており、将来は国際的な仕事に携わりたいと考えています。人とのコミュニケーションが得意で、チームでの活動やリーダーシップを取ることが好きです。今後も自分を磨き、社会に貢献できるよう努力していきたいと思っています。よろしくお願いします。


In [3]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
messages = [
SystemMessage("You are a helpful assistant."),
HumanMessage("こんにちは！私はジョンと言います！"),
AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
HumanMessage(content="私の名前がわかりますか？"),
]
ai_message = model.invoke(messages)
print(ai_message.content)

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？


In [5]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
messages = [
  SystemMessage("You are a helpful assistant."),
  HumanMessage("こんにちは！"),
]
for chunk in model.stream(messages):
  print(chunk.content, end="", flush=True)

こんにちは！どのようにお手伝いできますか？

In [8]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。
料理名: {dish}""")
prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

text='以下の料理のレシピを考えてください。\n料理名: カレー'


In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "ユーザーが入力した料理のレシピを考えてください。"),
    ("human", "{dish}"),
])

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [10]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
])

prompt_value = prompt.invoke({
    "chat_history": [
        HumanMessage(content="こんにちは！私はジョンと言います！"),
        AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    ],
    "input": "私の名前がわかりますか？",
})

print(prompt_value)


messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前がわかりますか？', additional_kwargs={}, response_metadata={})]


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    (
        "user",
        [
            {"type": "text", "text": "画像を説明してください。"},
            {"type": "image_url", "image_url": {"url": "{image_url}"}},
        ],
    ),
])

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"
prompt_value = prompt.invoke({"image_url": image_url})

model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

この画像は、本の表紙です。タイトルは「ChatGPT/ LangChainによるチャットシステム構築[実践]入門」で、著者は吉田真吾と大嶋勇樹です。表紙にはカラフルな鳥のイラストが描かれています。内容としては、大規模言語モデルを本番システムで活用するための基礎知識と実践的なハンズオンについて説明しているようです。OpenAI APIやLangChainの活用方法についても触れられています。


In [14]:
from pydantic import BaseModel, Field
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

from langchain_core.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
# print(format_instructions)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}",
    ),
    ("human", "{dish}"),
])

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)


=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
カレー


In [15]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、さらに炒める。",
    "水を加え、煮立ったらアクを取り除く。",
    "弱火にして、約20分煮込む。",
    "カレールーを加え、溶かしながらさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [16]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、約20分煮込む。', 'カレールーを加え、溶かしながらさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']
